О сервисе «СберАвтоподписка»
«СберАвтоподписка» предлагает новый для российского рынка способ владения автомобилем и выступает в качестве альтернативы автокредиту. Машина — это актив, который быстро теряет в цене, поэтому мы рекомендуем вкладывать собственные средства в финансовые инструменты, а автомобиль брать во временное пользование за комфортную сумму в месяц.

#### Задача

  - изучить предоставленный датасет,
  - ответить на вопросы из общей части (подразумевающей базовую обработку данных и их разведочный анализ)  
  - выполнить задание по специализации ML-инженер.

#### План работы 

1. Подготовительная часть
    - ознакомление с предоставленным датасетом и описание атрбиутов
    - оценка и описательные статистики данных
    - приведение данных к нормальному виду для дальнейшей работы
2. Разведочный анализ данных
    - базова чистка- проверка на дубликаты, пустые значения, типизация данных, ненужные атрибуты
    - распределения ключевых атрибутов и их отношение
3. Задание
    - предсказать cовершение целевого действия(ROC_AUC~0.65) - факт совершения пользователем целевого действия
    - упаковать модель в сервис, который будет брать на вход все атрибуты, типа utm_*, devace_*, geo_*, и отдавать на выход 0/1(1-совершит любое целевое
      деиствие)

#### Описание представленных данных

Описание атрибутов:

• session_id — ID визита - удалено  
• client_id — ID посетителя- удалено  
• visit_number — порядковы номер визита клиента - удалено  
• utm_campaign — рекламная компания  
• device_category — тип устройства  
• device_os — ОС устройства  
• device_brand — марка устройства  
• device_model — модель устройства  
• device_browser — браузер  
• geo_country — страна  
• day_of_the_week - день недели посещения сайта  
• quarter_of_the_year - в каком кватрале текущего года  
• viewing_time - время суток посещения  
• advert_social_media - пришел пользователь из социальной сети или нет  
• organic_traffic - органический трафик  
• visit_big_city - посещение из Москвы или Санкт-Петербурга  
• moskow_area - города Московской области  
• target_action - целевая переменная  

#### Импорт пакетов

In [29]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.calibration import CalibratedClassifierCV


In [30]:
#pwd

In [31]:
'''
import os
path = ''
os.chdir(path)
os.getcwd()
'''

"\nimport os\npath = ''\nos.chdir(path)\nos.getcwd()\n"

#### Загрузка данных

In [32]:
# в качестве индекса использую session_id
df_full = pd.read_csv('data/ga_full.csv', index_col='session_id', low_memory=False)
df_full.head()

,utm_campaign,utm_adcontent,device_category,device_os,device_brand,device_browser,geo_country,day_of_the_week,quarter_of_the_year,viewing_time,advert_social_media,organic_traffic,visit_big_city,moskow_area,target_action
session_id,,,,,,,,,,,,,,,
9055434745589932991.1637753792.1637753792,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,mobile,Android,Huawei,Chrome,Russia,Wednesday,4,day,0,0,0,0,0
905544597018549464.1636867290.1636867290,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,mobile,Android,Samsung,Samsung Internet,Russia,Sunday,4,morning,0,0,1,0,0
9055446045651783499.1640648526.1640648526,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,mobile,Android,Huawei,Chrome,Russia,Tuesday,4,night,0,0,0,0,0
9055447046360770272.1622255328.1622255328,gecBYcKZCPMcVYdSSzKP,other_content,mobile,Android,Xiaomi,Chrome,Russia,Saturday,2,night,0,0,1,0,0
9055447046360770272.1622255345.1622255345,gecBYcKZCPMcVYdSSzKP,other_content,mobile,Android,Xiaomi,Chrome,Russia,Saturday,2,night,0,0,1,0,0


In [33]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1732266 entries, 9055434745589932991.1637753792.1637753792 to 9055430416266113553.1640968742.1640968742
Data columns (total 15 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   utm_campaign         object
 1   utm_adcontent        object
 2   device_category      object
 3   device_os            object
 4   device_brand         object
 5   device_browser       object
 6   geo_country          object
 7   day_of_the_week      object
 8   quarter_of_the_year  int64 
 9   viewing_time         object
 10  advert_social_media  int64 
 11  organic_traffic      int64 
 12  visit_big_city       int64 
 13  moskow_area          int64 
 14  target_action        int64 
dtypes: int64(6), object(9)
memory usage: 211.5+ MB


In [34]:
# просмотр количества уникальных значений в данных
for col in df_full.columns:
    print(col, df_full[col].nunique())

utm_campaign 5
utm_adcontent 5
device_category 3
device_os 7
device_brand 5
device_browser 9
geo_country 2
day_of_the_week 7
quarter_of_the_year 3
viewing_time 4
advert_social_media 2
organic_traffic 2
visit_big_city 2
moskow_area 2
target_action 2


#### Разделение данных на обучающую и тестовую выбоки

In [35]:
X = df_full.drop('target_action', axis=1)
y = df_full['target_action']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

Xtrain.shape, Xtest.shape

((1212586, 14), (519680, 14))

#### Преобразование категориальных переменных с помощью OneHotEncoder

In [36]:
# выберу столюцы по типу данных
columns = X.select_dtypes(include=['object']).columns

# преобразую категориальные переменные
ohe = OneHotEncoder(sparse_output=False)

ohe_Xtrain = ohe.fit_transform(Xtrain[columns])
ohe_Xtest = ohe.transform(Xtest[columns])


In [37]:
# преобразую в датафрейм, session_id использую в качестве индекса для последующего объединения 
df_ohe_Xtrain = pd.DataFrame(ohe_Xtrain, index=Xtrain.index, columns=ohe.get_feature_names_out())
df_ohe_Xtest = pd.DataFrame(ohe_Xtest, index=Xtest.index, columns=ohe.get_feature_names_out())
df_ohe_Xtrain.shape, df_ohe_Xtest.shape

((1212586, 47), (519680, 47))

In [38]:
# добавлю закодированные признаки к обучающей и тестовой выборкам
Xtrain = Xtrain.join(df_ohe_Xtrain)
Xtest = Xtest.join(df_ohe_Xtest)

# удалю исходные столбцы с закодированными признаками
Xtrain.drop(columns=columns, inplace=True)
Xtest.drop(columns=columns, inplace=True)


In [39]:
Xtrain.head()

,quarter_of_the_year,advert_social_media,organic_traffic,visit_big_city,moskow_area,utm_campaign_FTjNLDyTrXaWYgZymFkV,utm_campaign_LEoPHuyFvzoNfnzGgfcd,utm_campaign_LTuZkdKfxRGVceoWkVyg,utm_campaign_gecBYcKZCPMcVYdSSzKP,utm_campaign_other_campaign,...,day_of_the_week_Monday,day_of_the_week_Saturday,day_of_the_week_Sunday,day_of_the_week_Thursday,day_of_the_week_Tuesday,day_of_the_week_Wednesday,viewing_time_day,viewing_time_evening,viewing_time_morning,viewing_time_night
session_id,,,,,,,,,,,,,,,,,,,,,
496389678626874679.1636638120.1636638120,4,0,0,1,0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7709960390992303162.1634571321.1634571321,4,1,0,1,0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2869148166651091422.1630771691.1630771691,3,0,0,1,0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3481774815615902409.1622228297.1622228297,2,0,1,1,0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6263956477081137962.1626190678.1626190678,3,0,0,1,0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


#### Стандартизация признаков

In [40]:
# столбец quarter_of_the_year показывает квартал текущего года
# сделаю стандартизацию для него

scaler = StandardScaler()

Xtrain['std_quarter_of_the_year'] = scaler.fit_transform(Xtrain[['quarter_of_the_year']])
Xtest['std_quarter_of_the_year'] = scaler.transform(Xtest[['quarter_of_the_year']])

# удалю исходный столбец
Xtrain.drop('quarter_of_the_year', inplace=True, axis=1)
Xtest.drop('quarter_of_the_year', inplace=True, axis=1)

In [41]:
Xtrain.shape, Xtest.shape

((1212586, 52), (519680, 52))

In [180]:
'''
# не используется!
# сделаю понижение размерности данных на int8 для для увеличения скорости обработки
Xtrain = Xtrain.astype('int8')
Xtest = Xtest.astype('int8')

# также для целевой переменной
ytrain = ytrain.astype('int8')
ytest = ytest.astype('int8')


Xtrain.dtypes
'''


"\n# не используется!\n# сделаю понижение размерности данных на int8 для для увеличения скорости обработки\nXtrain = Xtrain.astype('int8')\nXtest = Xtest.astype('int8')\n\n# также для целевой переменной\nytrain = ytrain.astype('int8')\nytest = ytest.astype('int8')\n\n\nXtrain.dtypes\n"

### Modeling

#### Логистическая регрессия

In [80]:
# логистическая регрессия со стандартными параметрами
logreg = LogisticRegression(class_weight='balanced', random_state=42)

logreg.fit(Xtrain, ytrain)

LogisticRegression(class_weight='balanced', random_state=42)

In [81]:
# точность accuracy на тестовой выборке
predict_test_logreg = logreg.predict(Xtest)
accuracy_score(ytest, predict_test_logreg)

0.5899495843596059

In [16]:
# метрика roc_auc на тестовой выборке
roc_auc_score(ytest, logreg.predict_proba(Xtest)[:,1])

0.6451170804799605

In [17]:
# параметры модели
logreg.get_params()

{'C': 1.0,
 'class_weight': 'balanced',
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'deprecated',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 42,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [18]:
# подбор оптимальных параметров для logreg
param_grid = {
    'C': [1, 2],
    'fit_intercept': [True, False],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga'],
    'max_iter': [100, 300, 500]
}

# модель для перебора параметров базовой модели
grid_search_logreg = GridSearchCV(
    estimator=logreg,
    param_grid=param_grid,
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1
)

# перебор параметров
grid_search_logreg.fit(Xtrain, ytrain)

# лучшие параметры
best_params_logreg = grid_search_logreg.best_params_
best_params_logreg

Fitting 5 folds for each of 48 candidates, totalling 240 fits


{'C': 2, 'fit_intercept': True, 'max_iter': 100, 'solver': 'liblinear'}

In [38]:
# метрика с наилучшими параметрами
grid_search_logreg.best_score_

0.6490560845219357

In [17]:
# логистистическая регрессия с оптимальными параметрами
logreg = LogisticRegression(class_weight='balanced', C=2, verbose=1, fit_intercept=True, max_iter=100, solver='liblinear', random_state=42)
logreg.fit(Xtrain, ytrain)

[LibLinear]

LogisticRegression(C=2, class_weight='balanced', random_state=42,
                   solver='liblinear', verbose=1)

In [18]:
# accuracy и roc_auc на тестовой выборке
predict_test_logreg = logreg.predict(Xtest)
print('accuracy', accuracy_score(ytest, predict_test_logreg))
print('roc_auc', roc_auc_score(ytest, logreg.predict_proba(Xtest)[:,1]))

accuracy 0.5816733374384236
roc_auc 0.6453618625023622


#### Случайный лес

In [31]:
rf = RandomForestClassifier(class_weight='balanced', random_state=42)

rf.fit(Xtrain, ytrain)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [32]:
# точность accuracy на тестовой выборке
predict_test_rf = rf.predict(Xtest)
accuracy_score(ytest, predict_test_rf)

0.7052185960591133

In [33]:
# метрика roc_auc на тестовой выборке
roc_auc_score(ytest, rf.predict_proba(Xtest)[:,1])

0.6082368762550486

In [34]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [40]:
# подбор оптимальных параметров для rf
param_rf =  {
    'n_estimators': list(range(100, 501, 100)),
    'min_samples_split': [3, 5],
    'criterion': ['gini', 'entropy'],
    'min_samples_leaf': [1, 3, 5]   
}

# модель для перебора параметров базовой модели
grid_search_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_rf,
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1
)

# перебор параметров
grid_search_rf.fit(Xtrain, ytrain)

# лучшие параметры
best_params_rf = grid_search_rf.best_params_
best_params_rf

Fitting 5 folds for each of 60 candidates, totalling 300 fits


{'criterion': 'gini',
 'min_samples_leaf': 5,
 'min_samples_split': 3,
 'n_estimators': 500}

In [ ]:
grid_search_rf.best_score_

0.639783641163928

In [ ]:
# модель случайный лес с оптимальными параметрами
rf = RandomForestClassifier(class_weight='balanced',criterion='gini', min_samples_leaf=5,
                             min_samples_split=3, n_estimators=500, random_state=42)

rf.fit(Xtrain, ytrain)

,n_estimators,500
,criterion,'gini'
,max_depth,None
,min_samples_split,3
,min_samples_leaf,5
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [44]:
# точность accuracy на тестовой выборке
predict_test_rf = rf.predict(Xtest)
accuracy_score(ytest, predict_test_rf)

0.6977890240147783

In [45]:
# метрика roc_auc на тестовой выборке
roc_auc_score(ytest, rf.predict_proba(Xtest)[:,1])

0.639559225996334

#### Модель дерево решений

In [21]:
clf = DecisionTreeClassifier(class_weight='balanced', random_state=42)
#calibrated_svc = CalibratedClassifierCV(svc, method='sigmoid', cv=5)

clf.fit(Xtrain, ytrain)

DecisionTreeClassifier(class_weight='balanced', random_state=42)

In [22]:
# точность accuracy на тестовой выборке
predict_test_knn = clf.predict(Xtest)
accuracy_score(ytest, predict_test_knn)

0.6302262931034482

In [23]:
# метрика roc_auc на тестовой выборке
roc_auc_score(ytest, clf.predict_proba(Xtest)[:,1])

0.6083455637512322

In [18]:
clf.get_params()

{'ccp_alpha': 0.0,
 'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': 42,
 'splitter': 'best'}

In [19]:
# подбор оптимальных параметров для slf
param_clf =  {
    'min_samples_leaf': [1, 3, 5],
    'min_samples_split': [3, 5, 7],
    'criterion': ['gini', 'entropy', 'log_loss'],
    
}

# модель для перебора параметров базовой модели
grid_search_clf = GridSearchCV(
    estimator=clf,
    param_grid=param_clf,
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1
)

# перебор параметров
grid_search_clf.fit(Xtrain, ytrain)

# лучшие параметры
best_params_clf = grid_search_clf.best_params_
best_params_clf

Fitting 5 folds for each of 27 candidates, totalling 135 fits


{'criterion': 'gini', 'min_samples_leaf': 5, 'min_samples_split': 3}

In [20]:
grid_search_clf.best_score_

0.6148904021846966

In [24]:
# модель clf с наилучшими параметрами
clf = DecisionTreeClassifier(criterion='gini', min_samples_leaf= 5, min_samples_split= 3, 
                             class_weight='balanced', random_state=42)

clf.fit(Xtrain, ytrain)

DecisionTreeClassifier(class_weight='balanced', min_samples_leaf=5,
                       min_samples_split=3, random_state=42)

In [25]:
# точность accuracy на тестовой выборке
predict_test_clf = clf.predict(Xtest)
accuracy_score(ytest, predict_test_clf)

0.6219923799261083

In [26]:
# метрика roc_auc на тестовой выборке
roc_auc_score(ytest, clf.predict_proba(Xtest)[:,1])

0.6138012751259047

#### Модель многослойный персептрон

In [28]:
mlp = MLPClassifier(random_state=42)

mlp.fit(Xtrain, ytrain)

MLPClassifier(random_state=42)

In [29]:
# точность accuracy на тестовой выборке
predict_test_mlp = mlp.predict(Xtest)
accuracy_score(ytest, predict_test_mlp)

0.9709552032019705

In [30]:
# метрика roc_auc на тестовой выборке
roc_auc_score(ytest, mlp.predict_proba(Xtest)[:,1])

0.6632762513436512

In [31]:
mlp.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 42,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [45]:
# подбор оптимальных параметров для mlp
param_mlp =  {
    'activation': ['identity', 'logistic','tanh', 'relu'],
    'max_iter': [100, 300, 500],
    'solver': ['sgd', 'adam'],
    'hidden_layer_sizes': [(100,50,20), (200,40)]
}

# модель для перебора параметров базовой модели
grid_search_mlp = GridSearchCV(
    estimator=mlp,
    param_grid=param_mlp,
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1
)

# перебор параметров
grid_search_mlp.fit(Xtrain, ytrain)

# лучшие параметры
best_params_mlp = grid_search_mlp.best_params_
best_params_mlp

Fitting 5 folds for each of 48 candidates, totalling 240 fits


{'activation': 'logistic',
 'hidden_layer_sizes': (100, 50, 20),
 'max_iter': 100,
 'solver': 'adam'}

In [46]:
# лучшая метрика 
grid_search_mlp.best_score_


0.671221284688891

In [47]:
# модель с наилучшими параметрами
mlp = MLPClassifier(hidden_layer_sizes=(100,50,20), max_iter=100, solver='adam', random_state=42, activation='logistic')
mlp.fit(Xtrain, ytrain)

MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50, 20),
              max_iter=100, random_state=42)

In [48]:
# точность accuracy на тестовой выборке
predict_test_mlp = mlp.predict(Xtest)
accuracy_score(ytest, predict_test_mlp)

0.9709552032019705

In [49]:
# метрика roc_auc на тестовой выборке
roc_auc_score(ytest, mlp.predict_proba(Xtest)[:,1])

0.6715806074413057

In [ ]:
'''
# обучение модели на всей выборке
mlp.fit(X, y)

'''

In [ ]:
'''
# сохраню модель в файл
with open('data/model.pickle', 'wb') as file:
    pickle.dump(mlp, file)
    
'''

#### Результаты

 По результатм моделирования использованы четыре модели,лучшая метрика roc_auc у модели MLPClassifier. Требуемоеое значение roc_auc метрики достигнуто, roc_auc=0.66

Выводы


1. В ходе работы были исследованы два датасета с данными по новому сервису "Сберавтоподписка". Данные были проверены, очищены, пропуски запонены.
- в качестве индекса для объединения использован session_id, так как каждый визит характеризуется одной сессией,
- на основе visit_date и visit_time были созданы новые признаки - это день недели посещения, квартал года и время суток дня,
- по  utm_source создан признак пришел ли клиент из социальных сетей advert_social_media
- по utm_medium создан признак привлечение клиента из бесплатных источников organic_traffic,
- также в geo_city около 60% посетителей из двух городов, поэтому добавлен соттветствующий признак visit_big_city,
- utm_keyword и device_model не являются информативнми и содержат очень много пропусков, поэтому удалены
- 
 
2. Обработка и кодирование данных
- для снижения размерности данных, признаки с большим числом уникальных значений при малых численных были обьединены соответствующие группы,
- для кодирование категориальных переменных использовано OneHotEncoder
- 

#### My Pipeline

In [17]:
import pandas as pd
import random

from  datetime import datetime

In [3]:
#предзагрузка данных
df_sessions = pd.read_csv('data/ga_sessions.csv', low_memory=False)
df_hits = pd.read_csv('data/ga_hits.csv', low_memory=False)

In [5]:

# функция удаляет столбцы, выполняется после предподготовки данных
def filter_data(df_full: pd.DataFrame) -> pd.DataFrame:
    columns_to_drop = [
        'utm_keyword',
        'device_model',
        'client_id',
        'visit_number',
        'visit_date',
        'visit_time',
        'utm_source',
        'utm_medium',
        'device_screen_resolution',
        'geo_city'
    ]
    return df_full.drop(columns_to_drop, errors='ignore', axis=1)


# функция для заполнения пропусков  df_sessions
def prepare_missing(df_sessions: pd.DataFrame) -> pd.DataFrame:
    
    df_sessions = df_sessions.copy()
    
    # обработка пропусков в столбце utm_source
    df_sessions.utm_source = df_sessions.utm_source.fillna('ZpYIoDJMcFzVoPFsHGJL')

    
    # обработка значений в столбце utm_medium
    df_sessions['utm_medium'] = df_sessions.apply(lambda x: df_sessions.utm_medium.mode()[0] if x['utm_medium'] == '(not set)'
                                                  else x['utm_medium'], axis=1)
    df_sessions['utm_medium'] = df_sessions.apply(lambda x: 'organic' if x['utm_medium'] == '(none)'
                                                  else x['utm_medium'], axis=1)


    # обработка пропусков в столбце device_os
    android_mobile_device = list(set(df_sessions.device_brand.values))
    android_mobile_device.remove("Apple") 
    
    df_sessions['device_os'] = df_sessions.apply(lambda x: 'Android' if pd.isna(x['device_os']) and x['device_category'] == 'mobile' 
                                                 and (x['device_brand'] in android_mobile_device) else x['device_os'], axis=1)
    df_sessions['device_os'] = df_sessions.apply(lambda x: 'IOS' if pd.isna(x['device_os']) and x['device_category'] == 'mobile' 
                                                 and (x['device_brand'] == 'Apple') else x['device_os'], axis=1)
    df_sessions['device_os'] = df_sessions.apply(lambda x: 'IOS' if x['device_os'] == 'iOS' else x['device_os'], axis=1)
    df_sessions['device_os'] = df_sessions.apply(lambda x: 'Apple' if pd.isna(x['device_os']) and 
                                                 (x['device_browser'] == 'Safari' or x['device_browser'] == 'Safari (in-app)') 
                                                 else x['device_os'], axis=1)
    df_sessions['device_os'] = df_sessions.apply(lambda x: 'Windows' if pd.isna(x['device_os']) and (x['device_category'] == 'desktop')
                                                 else x['device_os'], axis=1)
    df_sessions['device_os'] = df_sessions.apply(lambda x: 'Android' if pd.isna(x['device_os']) and (x['device_category'] == 'tablet')
                                                 else x['device_os'], axis=1)

    os = list(((df_sessions.device_os.value_counts(dropna=False) / df_sessions.shape[0]).index)[:6])
    df_sessions['device_os'] = df_sessions.apply(lambda x: 'other' if x['device_os'] == '(not set)' or 
                                                 x['device_os'] not in os else x['device_os'], axis=1)


    # обработка значений в столбце utm_campaign
    campaign = ['LTuZkdKfxRGVceoWkVyg', 'LEoPHuyFvzoNfnzGgfcd', 'FTjNLDyTrXaWYgZymFkV', 'gecBYcKZCPMcVYdSSzKP']
    df_sessions['utm_campaign'] = df_sessions.apply(lambda x: (random.choice(campaign)) if pd.isna(x['utm_campaign'])
                                                    else x['utm_campaign'], axis=1)
    df_sessions['utm_campaign'] = df_sessions.apply(lambda x: 'other_campaign' if x['utm_campaign'] not in 
                                                    ['LTuZkdKfxRGVceoWkVyg', 'LEoPHuyFvzoNfnzGgfcd', 'FTjNLDyTrXaWYgZymFkV', 'gecBYcKZCPMcVYdSSzKP'] 
                                                    else x['utm_campaign'], axis=1)

    
    # обработка значений в столбце utm_adcontent
    df_sessions['utm_adcontent'] = df_sessions['utm_adcontent'].ffill()
    content = list(((df_sessions.utm_adcontent.value_counts(dropna=False) / df_sessions.shape[0]).index)[:4])
    df_sessions['utm_adcontent'] = df_sessions.apply(lambda x: 'other_content' if x['utm_adcontent'] 
                                                     not in content else x['utm_adcontent'], axis=1)


    # обработка пропусков device_brand
    df_sessions['device_brand'] = df_sessions['device_brand'].fillna('other')
    df_sessions['device_brand'] = df_sessions.apply(lambda x: 'other' if x['device_brand'] == '(not set)'
                                                    else x['device_brand'], axis=1)
    device = list(((df_sessions.device_brand.value_counts(dropna=False) / df_sessions.shape[0]).index)[:5])
    df_sessions['device_brand'] = df_sessions.apply(lambda x: 'other' if x['device_brand'] 
                                                    not in device else x['device_brand'], axis=1)


    # обработка значений (not set) device_browser
    brouser_name = ['Chrome', 'Safari', 'Safari (in-app)', 'Android Webview', 'Samsung Internet', 'Opera', 'Firefox', 'Edge']

    df_sessions['device_browser'] = df_sessions.apply(lambda x: 'other' if x['device_browser'] not in 
                                                      brouser_name else x['device_browser'], axis=1)


    # обработка значений (not set) geo_country
    df_sessions['geo_country'] = df_sessions.apply(lambda x: df_sessions.geo_country.mode()[0] if x['geo_country'] == '(not set)'
                                                   else x['geo_country'], axis=1)
    df_sessions['geo_country'] = df_sessions.apply(lambda x: x['geo_country'] if x['geo_country'] == 'Russia' else 'other', axis=1)

    
    # обработка значений (not set) geo_city
    df_sessions['geo_city'] = df_sessions.apply(lambda x: 'other' if x['geo_city'] == '(not set)' else x['geo_city'], axis=1)

    return df_sessions
    


# функция обрабатывает признаки и создает новые из существующих для df_sessions
def create_feature(df_sessions: pd.DataFrame) -> pd.DataFrame:
    
    df_sessions = df_sessions.copy()

    # признаки на основе visit_date
    df_sessions['visit_date'] = pd.to_datetime(df_sessions.visit_date)
    df_sessions['day_of_the_week'] = df_sessions.visit_date.dt.day_name()
    df_sessions['quarter_of_the_year'] = df_sessions.visit_date.dt.quarter
    

    # признаки на основе visit_time
    def viewing_time(data):
        time = pd.to_datetime(data).strftime('%H:%M:%S')
        time = int(time.split(':')[0])
        if time > 0 and time <= 5:
            return 'night'
        elif time > 5 and time <= 10:
            return 'morning'
        elif time > 10 and time <= 18:
            return 'day'  
        return 'evening'
    df_sessions['viewing_time'] = df_sessions.apply(lambda x: viewing_time(x['visit_time']), axis=1)
    

    # признаки на основе utm_source
    social_media =  ['QxAxdyPLuQMEcrdZWdWb', 'MvfHsxlTijuriZxsqZqt', 
                     'ISrKoXQCxqqYvAZlCvjs', 'IZEXUFLARCUMynmHNBGo', 
                     'PlbkrSYoHuZBWfYjYnfw', 'gVRrcxiDQubJiljoTbGm']
    df_sessions['advert_social_media'] = df_sessions.utm_source.apply(lambda x: 1 if x in social_media else 0)


    # признаки на основе utm_medium
    df_sessions['organic_traffic'] = df_sessions.apply(lambda x: 1 if x['utm_medium'] == 'organic' or  x['utm_medium'] == 'referral' else 0, axis=1)
    

    # признаки на основе geo_city
    df_sessions['visit_big_city'] = df_sessions.apply(lambda x: 1 if x['geo_city'] == 'Moscow' or x['geo_city'] == 'Saint Petersburg' else 0, axis=1)
    moskow_region = ['Balashikha', 'Bronnitsy', 'Chernogolovka', 'Dmitrov', 'Dolgorudny', 'Domodedovo', 'Dubna', 'Elektrogorsk', 
                     'Elektrostal', 'Fryazino', 'Ivanteevka', 'Kashira', 'Klimovsk', 'Kolomna', 'Korolev', 'Kotelniki', 'Krasnogorsk', 
                     'Lytkarino', 'Lobnya', 'Losino-Petrovsky', 'Lukhovitsy', 'Lyubertsy', 'Mozhaysk', 'Mytishchi', 'Naro-Fominsk', 
                     'Noginsk', 'Odintsovo', 'Orekhovo-Zuyevo', 'Pavlovsky Posad', 'Podolsk', 'Pushkino', 'Ramenskoye', 'Reutov', 
                     'Rogovo', 'Ruza', 'Sergiyev Posad', 'Serpukhov', 'Solnechnogorsk','Stupino', 'Taldom', 'Vidnoye', 'Volokolamsk',
                     'Voskresensk', 'Yakhroma', 'Yegoryevsk', 'Zaraysk', 'Zvenigorod']
    df_sessions['moskow_area'] = df_sessions.apply(lambda x: 1 if x['geo_city'] in moskow_region else 0, axis=1)    

    return df_sessions


# окончательная сборка датасета, определение целевой переменной
def prepare_full(df_sessions: pd.DataFrame, df_hits: pd.DataFrame) -> pd.DataFrame:
    target = ['sub_car_claim_click', 'sub_car_claim_submit_click', 'sub_open_dialog_click', 
              'sub_custom_question_submit_click', 'sub_call_number_click', 'sub_callback_submit_click',
              'sub_submit_success', 'sub_car_request_submit_click']
    df_hits['target_action'] = df_hits.apply(lambda x: 1 if x['event_action'] in target  else 0, axis=1)
    target_action = df_hits.groupby('session_id')['target_action'].any().astype(int)
    df_full = pd.merge(left=df_sessions, right=target_action, on='session_id', how='inner')
    
    df_full = df_full.set_index('session_id')
    return df_full


def separation(df_sessions: pd.DataFrame, df_hits: pd.DataFrame):
    df = create_feature(prepare_missing(df_sessions))
    df_full = filter_data(prepare_full(df, df_hits))
    X = df_full.drop('target_action', axis=1)
    y = df_full['target_action']
    return X, y


In [64]:
#проверка функций
X, y = separation(df_sessions, df_hits)


In [65]:
X.head()

,utm_campaign,utm_adcontent,device_category,device_os,device_brand,device_browser,geo_country,day_of_the_week,quarter_of_the_year,viewing_time,advert_social_media,organic_traffic,visit_big_city,moskow_area
session_id,,,,,,,,,,,,,,
9055434745589932991.1637753792.1637753792,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,mobile,Android,Huawei,Chrome,Russia,Wednesday,4,day,0,0,0,0
905544597018549464.1636867290.1636867290,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,mobile,Android,Samsung,Samsung Internet,Russia,Sunday,4,morning,0,0,1,0
9055446045651783499.1640648526.1640648526,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,mobile,Android,Huawei,Chrome,Russia,Tuesday,4,night,0,0,0,0
9055447046360770272.1622255328.1622255328,FTjNLDyTrXaWYgZymFkV,other_content,mobile,Android,Xiaomi,Chrome,Russia,Saturday,2,night,0,0,1,0
9055447046360770272.1622255345.1622255345,gecBYcKZCPMcVYdSSzKP,other_content,mobile,Android,Xiaomi,Chrome,Russia,Saturday,2,night,0,0,1,0


In [66]:
y.head()

session_id
9055434745589932991.1637753792.1637753792    0
905544597018549464.1636867290.1636867290     0
9055446045651783499.1640648526.1640648526    0
9055447046360770272.1622255328.1622255328    0
9055447046360770272.1622255345.1622255345    0
Name: target_action, dtype: int32

In [57]:
# импорт библилотек
import joblib
import pandas as pd
import random
from  datetime import datetime

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


def main():
    print('Car Rental Predict Pipeline')

    df_sessions = pd.read_csv('data/ga_sessions.csv', low_memory=False)
    df_hits = pd.read_csv('data/ga_hits.csv', low_memory=False)

    #X = df_full.drop('target_action', axis=1)
    #y = df_full['target_action']
    X, y = separation(df_sessions, df_hits)
    
    numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X.select_dtypes(include=['object']).columns


    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(transformers=[
        ('numerical', numerical_transformer, numerical_features),
        ('categorical', categorical_transformer, categorical_features)
    ])

    
    models = (
        LogisticRegression(solver='liblinear', class_weight='balanced', C=2, verbose=1,
                           fit_intercept=True, max_iter=100),
        RandomForestClassifier(class_weight='balanced',criterion='gini', min_samples_leaf=5,
                               min_samples_split=3, n_estimators=500),
        DecisionTreeClassifier(criterion='gini', min_samples_leaf= 5, min_samples_split= 3,
                               class_weight='balanced'),
        MLPClassifier(activation='logistic', hidden_layer_sizes=(100,50,20), max_iter=100, solver='adam')
    )

    best_score = .0
    best_pipe = None
    for model in models:
        pipe = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', model)
        ])
        score = cross_val_score(pipe, X, y, cv=4, scoring='roc_auc')
        print(f'model: {type(model).__name__}, roc_mean: {score.mean():.4f}, roc_std: {score.std():.4f}')

        if score.mean() > best_score:
            best_score = score.mean()
            best_pipe = pipe

    best_pipe.fit(X, y)
    
    print(f'best model: {type(best_pipe.named_steps["classifier"]).__name__}, roc_auc: {best_score:.4f}')
    joblib.dump({
        'model': best_pipe,
        'metadata': {
            'name': 'Car rental predict model',
            'author': 'Alexey Slabinsky',
            'version': 1,
            'date': datetime.now(),
            'type': type(best_pipe.named_steps["classifier"]).__name__,
            'roc_auc': best_score
        }}, 'model/car_rental_pipe.pkl')


# Press the green button in the gutter to run the script.
if __name__ == '__main__':
    main()


Car Rental Predict Pipeline
[LibLinear][LibLinear][LibLinear][LibLinear]model: LogisticRegression, roc_mean: 0.6413, roc_std: 0.0026
model: RandomForestClassifier, roc_mean: 0.6333, roc_std: 0.0012
model: DecisionTreeClassifier, roc_mean: 0.6136, roc_std: 0.0013
model: MLPClassifier, roc_mean: 0.6587, roc_std: 0.0025
best model: MLPClassifier, roc_auc: 0.6587


In [162]:
# проверка предитка

In [143]:
d = pd.read_json('data/10092025.json', orient='index').T
d

,client_id,event_category,event_label,event_action,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,210838531.1636867225,pos_landing,gbMnjHwFWhPZfoQLyFat,go_to_car_card,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,lOMpyAuAISPFiAxjXIYM,385x854,Samsung Internet,Russia,Moscow


In [144]:
model = joblib.load('model/car_rental_pipe.pkl')
df = pd.read_json('data/11092025.json', orient='index').T

def predict(df):
    #
    client = df.client_id
    df = filter_data(create_feature(df))
    y = model['model'].predict(df)

    return {
        'Client_id': client[0],
        'Result': y[0]
    }


In [145]:
predict(df)

{'Client_id': '2108385564.1622255328', 'Result': 0}

In [146]:
model['metadata']

{'name': 'Car rental predict model',
 'author': 'Alexey Slabinsky',
 'version': 1,
 'date': datetime.datetime(2025, 9, 14, 6, 41, 43, 516201),
 'type': 'MLPClassifier',
 'roc_auc': 0.6587383599815197}

In [147]:
"""
def predict(form: Form):
    #
    df = pd.DataFrame.from_dict([form.model_dump()])
    y = model['model'].predict(filter_data(create_feature(df)))

    return {
        'client_id': df.client_id[0],
        'result': y[0]
    }

"""